In [1]:
from collections import defaultdict, deque

input_file = "input_files/day_10.txt"

with open(input_file) as lines:
    data = [l.strip() for l in lines]

In [2]:
N = [-1, 0]
S = [1, 0]
E = [0, 1]
W = [0, -1]

# col, row
pipes = {
    "|": (N, S),
    "-": (E, W),
    "L": (N, E),
    "J": (N, W),
    "7": (S, W),
    "F": (S, E)
}

def make_graph(data):
    '''
    Create a adjacency list map
    return the location of the start node and graph
    '''
    h, w = len(data), len(data[0])
    s = None
    symbols = defaultdict(list)
    s_connections = []
    
    # find the start to make the next
    # step easier since we don't know how S connects
    for row, line  in enumerate(data):
        for col, symbol in enumerate(line):
            if symbol == 'S':
                s = (row, col)

    for row, line  in enumerate(data):
        for col, symbol in enumerate(line):
            if symbol == '.' or symbol == 'S':
                continue
            for (r_d, c_d) in pipes[symbol]:
                con = (row + r_d, col + c_d)
                if con == s:
                    s_connections.append((row, col))
                symbols[(row, col)].append(con)

    symbols[s] = s_connections  
    return s, symbols

start, g = make_graph(data)

print("S location:", start)

S location: (17, 83)


## Part One
Breadth first travesal of the graph keeping track of the max depth

In [3]:
def bfs(g, start):
    seen = set()
    q = deque([(start,0)])
    max_depth = 0

    while len(q):
        node, depth = q.popleft()
        for next_node in g[node]:
            if next_node not in seen:
                seen.add(next_node)
                q.append((next_node, depth + 1))
                max_depth = max(depth + 1, max_depth)
                
    return max_depth
   
max_depth = bfs(g, start)
max_depth

7063

## Part Two

Find the ordered polygon based on the graph. For each coordinate not part of the polygon, use the winding rule to determine if it is inside or outside the boundary. It's probably more effience to count edges (ray crossing), but the case where "there doesn't even need to be a full tile path" is problematic.

In [4]:
def get_polygon(g, start, first):
    # pick a starting direction:
    curr_vertice = g[start][0]
    
    poly = [start, curr_vertice]
   
    # loop through graph picking the node we have not just traversed
    while curr_vertice != start:
        curr_vertice = next(p for p in g[poly[-1]] if p != poly[-2])
        poly.append(curr_vertice)
    return poly

p = get_polygon(g, start,  (17, 82))
len(p)

14127

In [5]:
from itertools import pairwise

def determinant(p, start, end):
    x, y = p
    x0, y0 = start
    x1, y1 = end
    return (x1 - x0) * (y - y0) - (y1 - y0) * (x - x0)

def get_winding_number(tile, polygon):
    windings = 0
    x, y = tile
    ys = [p[1] for p in polygon]
    
    for (y0, y1), (pol1, pol2) in zip(pairwise(ys), pairwise(polygon)):
        if y0 <= y:
            if y1 > y:
                if determinant(tile, pol1, pol2) > 0:
                    windings += 1
        elif y1 <= y:
            if determinant(tile, pol1, pol2) < 0:
                windings -= 1
    return windings

def get_tiles(data, poly):
    borders = set(poly)
    tiles = []
    for row, line in enumerate(data):
        for col, ch in enumerate(line):
            if (row, col) not in borders:
                tiles.append((row, col))
    return tiles


In [6]:
tiles = get_tiles(data, p)
sum(get_winding_number(tile, p) != 0 for tile in tiles)

589

# Part 2: counting edges with numpy is **much** faster
Take the count of edges from the point toward the left. Only count the '|' and do a little math to make sure the corner combinations are even for LJ and F7 combinations (they won't count) and odd for L7 and FJ combinations they do count as a block. 

Then find if the count is even or odd. Odd counts are inside. Cheating a bit and considering the `S` node a 'J' by inpecting the input.

In [7]:
import numpy as np

m = np.zeros([len(data), len(data[0])], dtype=int)

for x, y in p:
    char = data[x][y]
    if char == '|':
        m[x, y] = 1 
    elif char in '7F':
        m[x, y] = 1

cumsum = np.cumsum(m, axis=1)

# don't count the actual path
cumsum[tuple(zip(*p))] = 0

(cumsum % 2 == 1).sum()


589